# 🧪 LAB — Ficheros, Regex y Agregaciones

En este laboratorio trabajarás con un conjunto realista de datos compuestos por:

- **Ficheros CSV** con ventas diarias
- **Log de actividad** con eventos de usuarios
- **Regex** para extraer información clave
- **Agregaciones estadísticas** y KPIs
- **Visualización** final

Objetivo: construir un *pipeline* ETL completo combinando ficheros heterogéneos.

---
## 1️⃣ Preparación de ficheros de ejemplo


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from io import StringIO
%matplotlib inline

Path('lab8').mkdir(exist_ok=True)

# Crear CSV de ventas
for dia in range(1,6):
    df = pd.DataFrame({
        'fecha': [f'2024-01-0{dia}'] * 4,
        'producto': ['A','B','C','D'],
        'unidades': np.random.randint(5,20,4),
        'precio': np.random.randint(10,50,4)
    })
    df.to_csv(f'lab8/ventas_{dia}.csv', index=False)

# Crear log de actividad
log_text = """
INFO user=1 event=login time=120ms
INFO user=2 event=login time=90ms
WARN user=3 event=timeout time=200ms
ERROR user=1 event=crash time=350ms
INFO user=2 event=logout time=100ms
"""

with open('lab8/logs.txt', 'w') as f:
    f.write(log_text)

sorted(Path('lab8').iterdir())

---
## 2️⃣ Cargar todos los CSV de ventas

### Objetivo:
- Unir todos los CSV en un solo DataFrame
- Asegurarse de que la columna `fecha` es datetime


In [ ]:
# TU CÓDIGO AQUÍ


---
## 3️⃣ Procesar logs con expresiones regulares

El formato es:
```
LEVEL user=X event=Y time=Zms
```

### Objetivo:
- Extraer user, event, time
- Crear un DataFrame `df_logs`


In [ ]:
# TU CÓDIGO AQUÍ


---
## 4️⃣ Crear KPIs combinando ventas y logs

### Objetivos:
1. Total de ventas por día
2. Ingresos diarios (`unidades * precio`)
3. Tiempo medio de eventos por usuario
4. Número de errores por usuario

Crear DataFrames agregados:
- `ventas_diarias`
- `errores_por_usuario`
- `tiempo_medio_eventos`


In [ ]:
# TU CÓDIGO AQUÍ


---
## 5️⃣ Visualización

### Crea:
1. Gráfico de barras → ingresos diarios
2. Gráfico de líneas → unidades totales por día
3. Gráfico horizontal → errores por usuario


In [ ]:
# TU CÓDIGO AQUÍ


---
# ✅ Soluciones (ocultas)

<details>
<summary>Mostrar soluciones</summary>

### 🔹 2. Cargar todos los CSV
```python
import glob

df_ventas = pd.concat([
    pd.read_csv(r) for r in glob.glob('lab8/ventas_*.csv')
])
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'])
```

### 🔹 3. Logs con regex
```python
patron = re.compile(r'user=(\d+) event=(\w+) time=(\d+)ms')

datos = []
with open('lab8/logs.txt') as f:
    for linea in f:
        m = patron.search(linea)
        if m:
            datos.append({
                'user': int(m.group(1)),
                'event': m.group(2),
                'time_ms': int(m.group(3))
            })

df_logs = pd.DataFrame(datos)
```

### 🔹 4. KPIs
```python
df_ventas['ingresos'] = df_ventas['unidades'] * df_ventas['precio']

ventas_diarias = df_ventas.groupby('fecha').agg({
    'unidades': 'sum',
    'ingresos': 'sum'
})

tiempo_medio_eventos = df_logs.groupby('user')['time_ms'].mean()

errores_por_usuario = df_logs[df_logs['event']=='crash'].groupby('user').size()
```

### 🔹 5. Visualización
```python
ventas_diarias['ingresos'].plot.bar(title='Ingresos diarios')
plt.show()

ventas_diarias['unidades'].plot(title='Unidades por día')
plt.show()

errores_por_usuario.plot.barh(title='Errores por usuario')
plt.show()
```
</details>